In [1]:
# Importing Packages and Data
import pandas as pd
import numpy as np
import biogeme as biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msg
from biogeme.expressions import Beta
from biogeme.expressions import (
    Beta,
    DefineVariable,
    bioDraws,
    PanelLikelihoodTrajectory,
    MonteCarlo,
    log,
    Derive,
    bioNormalCdf,
    Elem
)
import math
from datetime import datetime
import matplotlib.pyplot as plt
from factor_analyzer import FactorAnalyzer

In [2]:
# Import psychometric variables
micro_psych = pd.read_csv('Data/Psych_Micro.csv')

In [3]:
# drop unique_ID column
micro_psych = micro_psych.drop('unique_id', axis=1)
micro_psych = micro_psych.drop('user', axis = 1)

In [4]:
# show NaNs in micro_psych
nan_counts = micro_psych.isna().sum()
print(nan_counts)


Unnamed: 0         0
zipn               0
p_x                0
dperson_x          0
sp                 0
                  ..
antimicro2         0
antimicro3         0
intentiontouse1    0
intentiontouse2    0
intentiontouse3    0
Length: 133, dtype: int64


In [5]:
# drop envbenefit4 since unused and contains NaNs
micro_psych = micro_psych.drop('envbenefit4', axis = 1)

In [6]:
# sort
micro_psych = micro_psych.sort_values(by=['who'])

# convert to database
database = db.Database('micro_psych', micro_psych)

database.panel("who")


globals().update(database.variables)

In [7]:
# Parameters to be estimated
# Should one of these = 1?

intercept_SN = Beta('intercept_SN', 0, None, None, 0)
coef_veh_SN = Beta('coef_veh_SN', 0, None, None, 0)
coef_bike_SN = Beta('coef_bike_SN', 0, None, None, 0)
coef_hhsize_SN = Beta('coef_hhsize_SN', 0, None, None, 0)
coef_stu_SN = Beta('coef_stu_SN', 0, None, None, 0)
omega_SN= bioDraws('omega_SN','NORMAL_HALTON3')
sigma_SN = Beta('sigma_SN',1,None,None,0)


intercept_EP = Beta('intercept_EP', 0, None, None, 0)
coef_veh_EP = Beta('coef_veh_EP', 0, None, None, 0)
coef_bike_EP = Beta('coef_veh_EP', 0, None, None, 0)
coef_hhsize_EP = Beta('coef_hhsize_EP', 0, None, None, 0)
coef_stu_EP = Beta('coef_stu_EP', 0, None, None, 0)
omega_EP = bioDraws('omega_EP', 'NORMAL_HALTON3')
sigma_EP = Beta('sigma_EP', 1, None, None, 0)


intercept_ATT = Beta('intercept_ATT', 0, None, None, 0)
coef_veh_ATT = Beta('coef_veh_ATT', 0, None, None, 0)
coef_bike_ATT = Beta('coef_bike_ATT', 0, None, None, 0)
coef_hhsize_ATT = Beta('coef_hhsize_ATT', 0, None, None, 0)
coef_stu_ATT = Beta('coef_stu_ATT', 0, None, None, 0)
omega_ATT = bioDraws('omega_ATT','NORMAL_HALTON3')
sigma_ATT = Beta('sigma_ATT',1, None, None ,0)



In [8]:
# STRUCTURAL EQUATIONS

SN = (
    intercept_SN
    + coef_veh_SN * veh_dummy
    + coef_bike_SN * bike
    + coef_hhsize_SN * hhsize
    + coef_stu_SN * stu_dummy
    + sigma_SN * omega_SN
)

EP = (
    intercept_EP
    + coef_veh_EP * veh_dummy
    + coef_bike_EP * bike
    + coef_hhsize_EP * hhsize
    + coef_stu_EP * stu_dummy
    + sigma_EP * omega_EP
)

ATT = (
    intercept_ATT
    + coef_veh_ATT * veh_dummy
    + coef_bike_ATT * bike
    + coef_hhsize_ATT * hhsize
    + coef_stu_ATT * stu_dummy
    + sigma_ATT * omega_ATT
)

In [9]:
B_SN01_F1 = Beta('B_SN01_F1',-1,None,None,1)
B_SN02_F1 = Beta('B_SN02_F1',-1,None,None,0)
B_SN03_F1 = Beta('B_SN03_F1',-1,None,None,0)

INTER_SN01 = Beta('INTER_SN01',0,None,None,1)
INTER_SN02 = Beta('INTER_SN02',0,None,None,0)
INTER_SN03 = Beta('INTER_SN03',0,None,None,0)

MODEL_SN01 = INTER_SN01 + B_SN01_F1 * SN
MODEL_SN02 = INTER_SN02 + B_SN02_F1 * SN
MODEL_SN03 = INTER_SN03 + B_SN03_F1 * SN

SIGMA_STAR_SN01 = Beta('SIGMA_STAR_SN01',1,1.0e-5,None,1)
SIGMA_STAR_SN02 = Beta('SIGMA_STAR_SN02',1,1.0e-5,None,0)
SIGMA_STAR_SN03 = Beta('SIGMA_STAR_SN03',1,1.0e-5,None,0)

delta_1 = Beta('delta_1',0.1, 1.0e-5, None, 1)
delta_2 = Beta('delta_2',0.2,1.0e-5, None, 0)
tau_1 = -delta_1 - delta_2
tau_2 = -delta_1 
tau_3 = 0
tau_4 = delta_1
tau_5 = delta_1 + delta_2

SN01_tau_1 = (tau_1-MODEL_SN01)/SIGMA_STAR_SN01
SN01_tau_2 = (tau_2-MODEL_SN01)/SIGMA_STAR_SN01
SN01_tau_3 = (tau_3-MODEL_SN01)/SIGMA_STAR_SN01
SN01_tau_4 = (tau_4-MODEL_SN01)/SIGMA_STAR_SN01
SN01_tau_5 = (tau_5-MODEL_SN01)/SIGMA_STAR_SN01

SN02_tau_1 = (tau_1-MODEL_SN02)/SIGMA_STAR_SN02
SN02_tau_2 = (tau_2-MODEL_SN02)/SIGMA_STAR_SN02
SN02_tau_3 = (tau_3-MODEL_SN02)/SIGMA_STAR_SN02
SN02_tau_4 = (tau_4-MODEL_SN02)/SIGMA_STAR_SN02
SN02_tau_5 = (tau_5-MODEL_SN02)/SIGMA_STAR_SN02

SN03_tau_1 = (tau_1-MODEL_SN03)/SIGMA_STAR_SN03
SN03_tau_2 = (tau_2-MODEL_SN03)/SIGMA_STAR_SN03
SN03_tau_3 = (tau_3-MODEL_SN03)/SIGMA_STAR_SN03
SN03_tau_4 = (tau_4-MODEL_SN03)/SIGMA_STAR_SN03
SN03_tau_5 = (tau_5-MODEL_SN03)/SIGMA_STAR_SN03

# Likert scale with M = 6 measurements
IndSN01 = {
    1: bioNormalCdf(SN01_tau_1),
    2: bioNormalCdf(SN01_tau_2)-bioNormalCdf(SN01_tau_1),
    3: bioNormalCdf(SN01_tau_3)-bioNormalCdf(SN01_tau_2),
    4: bioNormalCdf(SN01_tau_4)-bioNormalCdf(SN01_tau_3),
    5: bioNormalCdf(SN01_tau_5)-bioNormalCdf(SN01_tau_4),
    6: 1-bioNormalCdf(SN01_tau_5)
}
P_SN01 = Elem(IndSN01, socialnorm1)


IndSN02 = {
    1: bioNormalCdf(SN02_tau_1),
    2: bioNormalCdf(SN02_tau_2)-bioNormalCdf(SN02_tau_1),
    3: bioNormalCdf(SN02_tau_3)-bioNormalCdf(SN02_tau_2),
    4: bioNormalCdf(SN02_tau_4)-bioNormalCdf(SN02_tau_3),
    5: bioNormalCdf(SN02_tau_5)-bioNormalCdf(SN02_tau_4),
    6: 1-bioNormalCdf(SN02_tau_5)
}
P_SN02 = Elem(IndSN02, socialnorm2)

IndSN03 = {
    1: bioNormalCdf(SN03_tau_1),
    2: bioNormalCdf(SN03_tau_2)-bioNormalCdf(SN03_tau_1),
    3: bioNormalCdf(SN03_tau_3)-bioNormalCdf(SN03_tau_2),
    4: bioNormalCdf(SN03_tau_4)-bioNormalCdf(SN03_tau_3),
    5: bioNormalCdf(SN03_tau_5)-bioNormalCdf(SN03_tau_4),
    6: 1-bioNormalCdf(SN03_tau_5)
}
P_SN03 = Elem(IndSN03, socialnorm4) 


In [10]:
B_EP01_F1 = Beta('B_EP01_F1',-1,None,None,1)
B_EP02_F1 = Beta('B_EP02_F1',-1,None,None,0)
B_EP03_F1 = Beta('B_EP03_F1',-1,None,None,0)
B_EP04_F1 = Beta('B_EP04_F1',-1,None,None,0)
B_EP05_F1 = Beta('B_EP05_F1',-1,None,None,0)

INTER_EP01 = Beta('INTER_EP01',0,None,None,1)
INTER_EP02 = Beta('INTER_EP02',0,None,None,0)
INTER_EP03 = Beta('INTER_EP03',0,None,None,0)
INTER_EP04 = Beta('INTER_EP04',0,None,None,0)
INTER_EP05 = Beta('INTER_EP05',0,None,None,0)

MODEL_EP01 = INTER_EP01 + B_EP01_F1 * EP
MODEL_EP02 = INTER_EP02 + B_EP02_F1 * EP
MODEL_EP03 = INTER_EP03 + B_EP03_F1 * EP
MODEL_EP04 = INTER_EP04 + B_EP04_F1 * EP
MODEL_EP05 = INTER_EP05 + B_EP05_F1 * EP

SIGMA_STAR_EP01 = Beta('SIGMA_STAR_EP01',1,1.0e-5,None,1)
SIGMA_STAR_EP02 = Beta('SIGMA_STAR_EP02',1,1.0e-5,None,0)
SIGMA_STAR_EP03 = Beta('SIGMA_STAR_EP03',1,1.0e-5,None,0)
SIGMA_STAR_EP04 = Beta('SIGMA_STAR_EP04',1,1.0e-5,None,0)
SIGMA_STAR_EP05 = Beta('SIGMA_STAR_EP05',1,1.0e-5,None,0)

delta_1 = Beta('delta_1',0.1, 1.0e-5, None, 1)
delta_2 = Beta('delta_2',0.2,1.0e-5, None, 0)
tau_1 = -delta_1 - delta_2
tau_2 = -delta_1 
tau_3 = 0
tau_4 = delta_1
tau_5 = delta_1 + delta_2

EP01_tau_1 = (tau_1-MODEL_EP01)/SIGMA_STAR_EP01
EP01_tau_2 = (tau_2-MODEL_EP01)/SIGMA_STAR_EP01
EP01_tau_3 = (tau_3-MODEL_EP01)/SIGMA_STAR_EP01
EP01_tau_4 = (tau_4-MODEL_EP01)/SIGMA_STAR_EP01
EP01_tau_5 = (tau_5-MODEL_EP01)/SIGMA_STAR_EP01

EP02_tau_1 = (tau_1-MODEL_EP02)/SIGMA_STAR_EP02
EP02_tau_2 = (tau_2-MODEL_EP02)/SIGMA_STAR_EP02
EP02_tau_3 = (tau_3-MODEL_EP02)/SIGMA_STAR_EP02
EP02_tau_4 = (tau_4-MODEL_EP02)/SIGMA_STAR_EP02
EP02_tau_5 = (tau_5-MODEL_EP02)/SIGMA_STAR_EP02

EP03_tau_1 = (tau_1-MODEL_EP03)/SIGMA_STAR_EP03
EP03_tau_2 = (tau_2-MODEL_EP03)/SIGMA_STAR_EP03
EP03_tau_3 = (tau_3-MODEL_EP03)/SIGMA_STAR_EP03
EP03_tau_4 = (tau_4-MODEL_EP03)/SIGMA_STAR_EP03
EP03_tau_5 = (tau_5-MODEL_EP03)/SIGMA_STAR_EP03

EP04_tau_1 = (tau_1-MODEL_EP04)/SIGMA_STAR_EP04
EP04_tau_2 = (tau_2-MODEL_EP04)/SIGMA_STAR_EP04
EP04_tau_3 = (tau_3-MODEL_EP04)/SIGMA_STAR_EP04
EP04_tau_4 = (tau_4-MODEL_EP04)/SIGMA_STAR_EP04
EP04_tau_5 = (tau_5-MODEL_EP04)/SIGMA_STAR_EP04

EP05_tau_1 = (tau_1-MODEL_EP05)/SIGMA_STAR_EP05
EP05_tau_2 = (tau_2-MODEL_EP05)/SIGMA_STAR_EP05
EP05_tau_3 = (tau_3-MODEL_EP05)/SIGMA_STAR_EP05
EP05_tau_4 = (tau_4-MODEL_EP05)/SIGMA_STAR_EP05
EP05_tau_5 = (tau_5-MODEL_EP05)/SIGMA_STAR_EP05

IndEP01 = {
    1: bioNormalCdf(EP01_tau_1),
    2: bioNormalCdf(EP01_tau_2)-bioNormalCdf(EP01_tau_1),
    3: bioNormalCdf(EP01_tau_3)-bioNormalCdf(EP01_tau_2),
    4: bioNormalCdf(EP01_tau_4)-bioNormalCdf(EP01_tau_3),
    5: bioNormalCdf(EP01_tau_5)-bioNormalCdf(EP01_tau_4),
    6: 1-bioNormalCdf(EP01_tau_5)
}
P_EP01 = Elem(IndEP01, expectedperf1)


IndEP02 = {
    1: bioNormalCdf(EP02_tau_1),
    2: bioNormalCdf(EP02_tau_2)-bioNormalCdf(EP02_tau_1),
    3: bioNormalCdf(EP02_tau_3)-bioNormalCdf(EP02_tau_2),
    4: bioNormalCdf(EP02_tau_4)-bioNormalCdf(EP02_tau_3),
    5: bioNormalCdf(EP02_tau_5)-bioNormalCdf(EP02_tau_4),
    6: 1-bioNormalCdf(EP02_tau_5)
}
P_EP02 = Elem(IndEP02, expectedperf2)

IndEP03 = {
    1: bioNormalCdf(EP03_tau_1),
    2: bioNormalCdf(EP03_tau_2)-bioNormalCdf(EP03_tau_1),
    3: bioNormalCdf(EP03_tau_3)-bioNormalCdf(EP03_tau_2),
    4: bioNormalCdf(EP03_tau_4)-bioNormalCdf(EP03_tau_3),
    5: bioNormalCdf(EP03_tau_5)-bioNormalCdf(EP03_tau_4),
    6: 1-bioNormalCdf(EP03_tau_5)
}
P_EP03 = Elem(IndEP03, expectedperf3) 

IndEP04 = {
    1: bioNormalCdf(EP04_tau_1),
    2: bioNormalCdf(EP04_tau_2)-bioNormalCdf(EP04_tau_1),
    3: bioNormalCdf(EP04_tau_3)-bioNormalCdf(EP04_tau_2),
    4: bioNormalCdf(EP04_tau_4)-bioNormalCdf(EP04_tau_3),
    5: bioNormalCdf(EP04_tau_5)-bioNormalCdf(EP04_tau_4),
    6: 1-bioNormalCdf(EP04_tau_5)
}
P_EP04 = Elem(IndEP04, expectedperf4) 

IndEP05 = {
    1: bioNormalCdf(EP05_tau_1),
    2: bioNormalCdf(EP05_tau_2)-bioNormalCdf(EP05_tau_1),
    3: bioNormalCdf(EP05_tau_3)-bioNormalCdf(EP05_tau_2),
    4: bioNormalCdf(EP05_tau_4)-bioNormalCdf(EP05_tau_3),
    5: bioNormalCdf(EP05_tau_5)-bioNormalCdf(EP05_tau_4),
    6: 1-bioNormalCdf(EP05_tau_5)
}
P_EP05 = Elem(IndEP05, expectedperf5) 

In [11]:
B_ATT01_F1 = Beta('B_ATT01_F1',-1,None,None,1)
B_ATT02_F1 = Beta('B_ATT02_F1',-1,None,None,0)
B_ATT03_F1 = Beta('B_ATT03_F1',-1,None,None,0)

INTER_ATT01 = Beta('INTER_ATT01',0,None,None,1)
INTER_ATT02 = Beta('INTER_ATT02',0,None,None,0)
INTER_ATT03 = Beta('INTER_ATT03',0,None,None,0)

MODEL_ATT01 = INTER_ATT01 + B_ATT01_F1 * ATT
MODEL_ATT02 = INTER_ATT02 + B_ATT02_F1 * ATT
MODEL_ATT03 = INTER_ATT03 + B_ATT03_F1 * ATT

SIGMA_STAR_ATT01 = Beta('SIGMA_STAR_ATT01',1,1.0e-5,None,1)
SIGMA_STAR_ATT02 = Beta('SIGMA_STAR_ATT02',1,1.0e-5,None,0)
SIGMA_STAR_ATT03 = Beta('SIGMA_STAR_ATT03',1,1.0e-5,None,0)

delta_1 = Beta('delta_1',0.1, 1.0e-5, None, 1)
delta_2 = Beta('delta_2',0.2,1.0e-5, None, 0)
tau_1 = -delta_1 - delta_2
tau_2 = -delta_1 
tau_3 = 0
tau_4 = delta_1
tau_5 = delta_1 + delta_2

ATT01_tau_1 = (tau_1-MODEL_ATT01)/SIGMA_STAR_ATT01
ATT01_tau_2 = (tau_2-MODEL_ATT01)/SIGMA_STAR_ATT01
ATT01_tau_3 = (tau_3-MODEL_ATT01)/SIGMA_STAR_ATT01
ATT01_tau_4 = (tau_4-MODEL_ATT01)/SIGMA_STAR_ATT01
ATT01_tau_5 = (tau_5-MODEL_ATT01)/SIGMA_STAR_ATT01

ATT02_tau_1 = (tau_1-MODEL_ATT02)/SIGMA_STAR_ATT02
ATT02_tau_2 = (tau_2-MODEL_ATT02)/SIGMA_STAR_ATT02
ATT02_tau_3 = (tau_3-MODEL_ATT02)/SIGMA_STAR_ATT02
ATT02_tau_4 = (tau_4-MODEL_ATT02)/SIGMA_STAR_ATT02
ATT02_tau_5 = (tau_5-MODEL_ATT02)/SIGMA_STAR_ATT02

ATT03_tau_1 = (tau_1-MODEL_ATT03)/SIGMA_STAR_ATT03
ATT03_tau_2 = (tau_2-MODEL_ATT03)/SIGMA_STAR_ATT03
ATT03_tau_3 = (tau_3-MODEL_ATT03)/SIGMA_STAR_ATT03
ATT03_tau_4 = (tau_4-MODEL_ATT03)/SIGMA_STAR_ATT03
ATT03_tau_5 = (tau_5-MODEL_ATT03)/SIGMA_STAR_ATT03

# Likert scale with M = 6 measurements
IndATT01 = {
    1: bioNormalCdf(ATT01_tau_1),
    2: bioNormalCdf(ATT01_tau_2)-bioNormalCdf(ATT01_tau_1),
    3: bioNormalCdf(ATT01_tau_3)-bioNormalCdf(ATT01_tau_2),
    4: bioNormalCdf(ATT01_tau_4)-bioNormalCdf(ATT01_tau_3),
    5: bioNormalCdf(ATT01_tau_5)-bioNormalCdf(ATT01_tau_4),
    6: 1-bioNormalCdf(ATT01_tau_5)
}
P_ATT01 = Elem(IndATT01, antimicro1)


IndATT02 = {
    1: bioNormalCdf(ATT02_tau_1),
    2: bioNormalCdf(ATT02_tau_2)-bioNormalCdf(ATT02_tau_1),
    3: bioNormalCdf(ATT02_tau_3)-bioNormalCdf(ATT02_tau_2),
    4: bioNormalCdf(ATT02_tau_4)-bioNormalCdf(ATT02_tau_3),
    5: bioNormalCdf(ATT02_tau_5)-bioNormalCdf(ATT02_tau_4),
    6: 1-bioNormalCdf(ATT02_tau_5)
}
P_ATT02 = Elem(IndATT02, antimicro2)

IndATT03 = {
    1: bioNormalCdf(ATT03_tau_1),
    2: bioNormalCdf(ATT03_tau_2)-bioNormalCdf(ATT03_tau_1),
    3: bioNormalCdf(ATT03_tau_3)-bioNormalCdf(ATT03_tau_2),
    4: bioNormalCdf(ATT03_tau_4)-bioNormalCdf(ATT03_tau_3),
    5: bioNormalCdf(ATT03_tau_5)-bioNormalCdf(ATT03_tau_4),
    6: 1-bioNormalCdf(ATT03_tau_5)
}
P_ATT03 = Elem(IndATT03, antimicro3) 


In [12]:
ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 1)
ASC_SCOOTER_S = Beta('ASC_SCOOTER_S', 1, None, None, 0)
ASC_SCOOTER_RND = ASC_SCOOTER + ASC_SCOOTER_S * bioDraws('ASC_SCOOTER_RND', 'NORMAL_ANTI')

ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')

B_SCOOTERTIME = Beta('B_SCOOTERTIME', 0, None, None, 0)
B_SCOOTERTIME_PRIME = Beta('B_SCOOTERTIME_PRIME', 0, None, None, 0)
B_BIKETIME = Beta('B_BIKETIME', 0, None, None, 0)
B_SCTRANSITTIME = Beta('B_SCTRANSITTIME', 0, None, None, 0)

B_ACCESS = Beta('B_ACCESS', 0, None, None, 0)
B_DROP =Beta('B_DROP', 0, None, None, 0)
B_WAITAV = Beta('B_WAITAV', 0, None, None, 0)
B_AV = Beta('B_AV', 0, None, None, 0)

B_COST = Beta('B_COST', 0, None, None, 0)
B_COST_PRIME = Beta('B_COST_PRIME', 0, None, None, 0)
B_costadj = Beta('B_costadj', 0, None, None, 0)

B_PRCP_Yes = Beta('B_PRCP_Yes', 0, None, None, 0)
B_OWNBIKE = Beta('B_OWNBIKE', 0, None, None, 0)
B_BIKELANE = Beta('B_BIKELANE', 0, None, None, 0)
B_AGE_SQ = Beta('B_AGE_SQ', 0, None, None, 0)
B_AGE_SCTRANSIT_SQ = Beta('B_AGE_SCTRANSIT_SQ', 0, None, None, 0)
B_GENDER_F = Beta('B_GENDER_F', 0, None, None, 0)

B_AGE_MORE_35 = Beta('B_AGE_MORE_35', 0, None, None, 0)
B_CHILD_IN_HH = Beta('B_CHILD_IN_HH', 0, None, None, 0)
B_IS_WHITE = Beta('B_IS_WHITE', 0, None, None, 0)
B_FULLTIME_EMPLOY = Beta('B_FULLTIME_EMPLOY', 0, None, None, 0)
B_HHINCOME_MORE_75k = Beta('B_HHINCOME_MORE_75k', 0, None, None, 0)

B_EP_BIKE = Beta('B_EP_BIKE', 0, None, None, 0)
B_ATT_BIKE = Beta('B_ATT_BIKE', 0, None, None, 0)
B_SN_BIKE = Beta('B_SN_BIKE', 0, None, None, 0)

B_EP_SCTRANS = Beta('B_EP_SCTRANS', 0, None, None, 0)
B_ATT_SCTRANS = Beta('B_ATT_SCTRANS', 0, None, None, 0)
B_SN_SCTRANS = Beta('B_SN_BIKE', 0, None, None, 0)


In [13]:
V7 = (ASC_SCOOTER_RND
      + B_SCOOTERTIME*sctime
      + B_COST*sccost_adj
      + B_ACCESS*SCAW*(1-AVtech)
      + B_WAITAV*SCAV*AVtech
#     + ASC_SCOOTER_PRIME*AVtech
#     + B_SCOOTERTIME_PRIME*sctime*AVtech
#     + B_COST_PRIME*sccost_adj*AVtech
      )

V8 = (ASC_DLBIKE_RND
      + B_BIKETIME*dltime
      + B_COST*dlcost_adj
      + B_ACCESS*DLAW
      + B_ATT_BIKE*ATT
      + B_EP_BIKE*EP
      + B_SN_BIKE*SN       
    )

V9 = (ASC_DKBIKE_RND 
      + B_BIKETIME*dbtime 
      + B_COST*dbcost_adj
      + B_ACCESS*DBAW
      + B_DROP*DBDW
      + B_ATT_BIKE*ATT
      + B_EP_BIKE*EP
      + B_SN_BIKE*SN
    )

V10 = (ASC_SCTRANSIT_RND
       + B_SCTRANSITTIME*sttotime
       + B_COST*sttocost_adj 
       + B_ACCESS*STAW*(1-AVtech)
       + B_WAITAV*STAV*AVtech 
       + B_ATT_SCTRANS*ATT
      + B_EP_SCTRANS*EP
      + B_SN_SCTRANS*SN
)

In [14]:
V = {7: V7, 8: V8, 9: V9, 10: V10}
av = {7: scooter_av, 8: dlbike_av, 9: dkbike_av, 10: sctransit_av}
      

############################################### DO WE NEED SEPARATE COND PROB?
condprob = models.logit(V, av, choice)
condlike = (
    P_ATT01 * 
    P_ATT02 *
    P_ATT03 *
    P_EP01 *
    P_EP02 *
    P_EP03 *
    P_EP04 *
    P_EP05 *
    P_SN01 *
    P_SN02 *
    P_SN03 *
    condprob
)

condlikeindiv = PanelLikelihoodTrajectory(condlike)
logprob = log(MonteCarlo(condlikeindiv))


In [ ]:


the_biogeme = bio.BIOGEME(database, logprob, numberOfDraws=500, numberOfThreads=100)
the_biogeme.modelName = "HybridChoice_ATT_SN_EP_500Draws"

# Estimate the parameters
results = the_biogeme.estimate()
print(f"Estimated betas: {len(results.data.betaValues)}")
print(f"Final log likelihood: {results.data.logLike:.3f}")

In [ ]:
print(results)

In [ ]:
pandasResults = results.getEstimatedParameters()
print(pandasResults.round(2))